In [42]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [43]:
from selenium.webdriver.common.by import By

In [59]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

def get_jobs(num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    # Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')
    
    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.occupationParam=%22data+scientist%22&sc.locationSeoString=India&locId=115&locT=N'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  # If true, should be still looking for new jobs.

        # Let the page load. Change this number based on your internet speed.
        # Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        # Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME, "selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(0.1)

        try:
            driver.find_element(By.CLASS_NAME, "ModalStyle__xBtn___29PT9").click()  # Clicking the X.
        except NoSuchElementException:
            pass

        # Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, "jl")  # jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  # You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element(By.XPATH, './/div[@class="EmployerProfile_employerName__qujuA"]').text
                    location = driver.find_element(By.XPATH, './/div[@class="JobDetails_location__mSg5h"]').text
                    job_title = driver.find_element(By.XPATH, './/div[contains(@class, "JobDetails_jobTitle__Xvsha")]').text
                    job_description = driver.find_element(By.XPATH, './/div[@class="JobDetails_jobDescription__uW_fK"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element(By.XPATH, './/span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1  # You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element(By.XPATH, './/span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1  # You need to set a "not found value. It's important."

            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            # Going to the Company tab...
            # clicking on this:
            # <div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element(By.XPATH, './/div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    # <div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    # </div>
                    headquarters = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element(By.XPATH, './/div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  # Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            # add job to jobs

        # Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, './/li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching the target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  # This line converts the dictionary object into a pandas DataFrame.


In [60]:
# chrome_driver_path = "C:/Users/gaura/OneDrive/Desktop/Projects/DS_Salary/chromedriver"

In [61]:
#This line will open a new chrome window and start the scraping.
df = get_jobs(1, False)
df

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".selected"}
  (Session info: chrome=122.0.6261.95); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF664BEAD22+56930]
	(No symbol) [0x00007FF664B5F622]
	(No symbol) [0x00007FF664A142E5]
	(No symbol) [0x00007FF664A598ED]
	(No symbol) [0x00007FF664A59A2C]
	(No symbol) [0x00007FF664A9A967]
	(No symbol) [0x00007FF664A7BCDF]
	(No symbol) [0x00007FF664A981E2]
	(No symbol) [0x00007FF664A7BA43]
	(No symbol) [0x00007FF664A4D438]
	(No symbol) [0x00007FF664A4E4D1]
	GetHandleVerifier [0x00007FF664F66AAD+3709933]
	GetHandleVerifier [0x00007FF664FBFFED+4075821]
	GetHandleVerifier [0x00007FF664FB817F+4043455]
	GetHandleVerifier [0x00007FF664C89756+706710]
	(No symbol) [0x00007FF664B6B8FF]
	(No symbol) [0x00007FF664B66AE4]
	(No symbol) [0x00007FF664B66C3C]

In [46]:
df

NameError: name 'df' is not defined

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import pandas as pd
import time

def fetch_jobs(keyword, num_pages):
    options = Options()
    options.add_argument("window-size=1920,1080")
    
    # Enter your chromedriver.exe path below
    # chrome_path = r"C:\Users\Admin\Desktop\Work\Selenium\scraping-glassdoor-selenium-master\chromedriver.exe"
    
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.glassdoor.co.in/Job/Home/recentActivity.htm")
    
    search_input = driver.find_element(By.ID, "sc.keyword")
    search_input.send_keys(keyword)
    search_input.send_keys(Keys.ENTER)
    time.sleep(2)

    company_name = []
    job_title = []
    salary_est = []
    location = []
    job_description = []
    salary_estimate = []
    company_size = []
    company_type = []
    company_sector = []
    company_industry = []
    company_founded = []
    company_revenue = []
    
    # Set current page to 1
    current_page = 1     

    time.sleep(3)
    
    while current_page <= num_pages:   
        done = False
        while not done:
            job_cards = driver.find_elements(By.XPATH, "//article[@id='MainCol']//ul/li[@data-adv-type='GENERAL']")
            for card in job_cards:
                card.click()
                time.sleep(1)

                # Closes the signup prompt
                try:
                    driver.find_element(By.XPATH, ".//span[@class='SVGInline modal_closeIcon']").click()
                    time.sleep(2)
                except NoSuchElementException:
                    time.sleep(2)
                    pass

                # Expands the Description section by clicking on Show More
                try:
                    driver.find_element(By.XPATH, "//div[@class='css-t3xrds e856ufb2']").click()
                    time.sleep(1)
                except NoSuchElementException:
                    card.click()
                    print(str(current_page) + '#ERROR: no such element')
                    time.sleep(30)
                    driver.find_element(By.XPATH, "//div[@class='css-t3xrds e856ufb2']").click()
                except ElementNotInteractableException:
                    card.click()
                    driver.implicitly_wait(30)
                    print(str(current_page) + '#ERROR: not interactable')
                    driver.find_element(By.XPATH, "//div[@class='css-t3xrds e856ufb2']").click()

                # Scrape 
                try:
                    company_name.append(driver.find_element(By.XPATH, "//div[@class='css-xuk5ye e1tk4kwz5']").text)
                except:
                    company_name.append("#N/A")
                    pass

                try:
                    job_title.append(driver.find_element(By.XPATH, "//div[@class='css-1j389vi e1tk4kwz2']").text)
                except:
                    job_title.append("#N/A")
                    pass

                try:
                    location.append(driver.find_element(By.XPATH, "//div[@class='css-56kyx5 e1tk4kwz1']").text)
                except:
                    location.append("#N/A")
                    pass

                try:
                    job_description.append(driver.find_element(By.XPATH, "//div[@id='JobDescriptionContainer']").text)
                except:
                    job_description.append("#N/A")
                    pass

                try:
                    salary_estimate.append(driver.find_element(By.XPATH, "//div[@class='css-y2jiyn e2u4hf18']").text)
                except:
                    salary_estimate.append("#N/A")
                    pass
                
                try:
                    company_size.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Size']//following-sibling::*").text)
                except:
                    company_size.append("#N/A")
                    pass
                
                try:
                    company_type.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Type']//following-sibling::*").text)
                except:
                    company_type.append("#N/A")
                    pass
                    
                try:
                    company_sector.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Sector']//following-sibling::*").text)
                except:
                    company_sector.append("#N/A")
                    pass
                    
                try:
                    company_industry.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Industry']//following-sibling::*").text)
                except:
                    company_industry.append("#N/A")
                    pass
                    
                try:
                    company_founded.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Founded']//following-sibling::*").text)
                except:
                    company_founded.append("#N/A")
                    pass
                    
                try:
                    company_revenue.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Revenue']//following-sibling::*").text)
                except:
                    company_revenue.append("#N/A")
                    pass
                    
                done = True

       # Moves to the next page         
        if done:
            print(str(current_page) + ' ' + 'out of' +' '+ str(num_pages) + ' ' + 'pages done')
            driver.find_element(By.XPATH, "//span[@alt='next-icon']").click()   
            current_page = current_page + 1
            time.sleep(4)

    driver.close()
    df = pd.DataFrame({'company': company_name, 
    'job title': job_title,
    'location': location,
    'job description': job_description,
    'salary estimate': salary_estimate,
    'company_size': company_size,
    'company_type': company_type,
    'company_sector': company_sector,
    'company_industry' : company_industry, 'company_founded' : company_founded, 'company_revenue': company_revenue})
    
    df.to_csv(keyword + '.csv')

fetch_jobs("data scientist", 5)


KeyboardInterrupt: 

In [63]:
pip install requests 


  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   --------------- ----------------------- 41.0/100.4 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 100.4/100.4 kB 1.5 MB/s eta 0:00:00


In [65]:
pip install bs4

  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   ---------- ---------------------------- 41.0/147.9 kB 653.6 kB/s eta 0:00:01
   -------------------------------------- - 143.4/147.9 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 1.3 MB/s eta 0:00:00
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [66]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the URL
url = 'https://www.glassdoor.com/Job/jobs.htm?sc.occupationParam=%22data+scientist%22&sc.locationSeoString=India&locId=115&locT=N'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# Initialize lists to store the extracted data
job_title = []
salary_estimate = []
job_description = []
rating = []
company_name = []
location = []
headquarters = []
size = []
founded = []
type_of_ownership = []
industry = []
sector = []
revenue = []

# Extract job details
job_cards = soup.find_all('li', class_='react-job-listing')

for card in job_cards:
    title = card.find('a', class_='jobLink')
    if title:
        job_title.append(title.text.strip())
    else:
        job_title.append(None)
    
    salary = card.find('span', class_='css-1imh2hq e1wijj242')
    if salary:
        salary_estimate.append(salary.text.strip())
    else:
        salary_estimate.append(None)

    description = card.find('div', class_='jobInfoItem jobEmpolyerName')
    if description:
        job_description.append(description.text.strip())
    else:
        job_description.append(None)

    # Similarly extract other details such as rating, company name, location, etc.

# Now you can create a pandas DataFrame or any other data structure to store the extracted data
# Remember to handle None values or missing data appropriately

# Print or return the dataframe
# Example print statement:
print(job_title, salary_estimate, job_description, rating, company_name, location, headquarters, size, founded, type_of_ownership, industry, sector, revenue)


[] [] [] [] [] [] [] [] [] [] [] [] []
